In [50]:
import openai
import pandas as pd
from neo4j import GraphDatabase
from getpass import getpass
import json

In [4]:
openai_key = getpass()

 ········


In [27]:
neo4j_pwd = getpass()

 ········


In [7]:
openai.api_key = openai_key

In [31]:
bolt_uri = "bolt://44.204.43.21:7687"
driver = GraphDatabase.driver(bolt_uri, auth=("neo4j", neo4j_pwd))

In [87]:
def get_cypher(question):
    messages = [
        {"role": "system", "content": "You convert questions into Cypher queries for a Neo4j database"},
        {"role": "user", "content": "What was the score team when Germany played Canada?"},
        {"role": "assistant", "content": "MATCH (:Team {name: 'Germany'})-[sg:PLAYED_IN]->(match:Match)<-[sc:PLAYED_IN]-(:Team {name: 'Canada'}) RETURN sg.score AS GermanyScore, sc.score AS CanadaScore"},
        {"role": "user", "content": "Which teams played against France?"},
        {"role": "assistant", "content": "MATCH (:Team {name: 'France'})-[:PLAYED_IN]->()<-[:PLAYED_IN]-(t:Team) RETURN DISTINCT t.name AS teamName"},
        {"role": "user", "content": "Which team scored the most goals against Sweden?"},
        {"role": "assistant", "content": "MATCH (:Team {name: 'Sweden'})-[:PLAYED_IN]->(match:Match)<-[s:PLAYED_IN]-(t:Team) RETURN t.name AS teamName, s.score AS score ORDER BY s.score DESC LIMIT 1"},
        {"role": "user", "content": question}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    query = response['choices'][0]['message']['content']
    return query

In [91]:
def get_neo4j_records(query):
    with driver.session() as session:
        result = session.run(query)
        records = [row.data() for row in result]
    return records

In [105]:
def format_answer(records, question):
    messages = [
        {"role": "system", "content": f"Use the JSON data provided by the user to answer to the question."},
        {"role": "user", "content": "Question: Which team scored the fewest goals against Japan? JSON: [{'teamName': 'Switzerland', 'score': 0}]"},
        {"role": "assistant", "content": "Switzerland scored the fewest goals against Japan."},
        {"role": "user", "content": "What were the scores when Japan played Sweden? JSON: [{'JapanScore': 3, 'SwedenScore': 1}, {'JapanScore': 0, 'SwedenScore': 8}, {'JapanScore': 0, 'SwedenScore': 2}]"},
        {"role": "assistant", "content": "The score when Japan played Sweden was as follows: \n- Japan: 3, Sweden: 1\n- Japan: 0, Sweden: 8,\n- Japan: 0, Sweden: 2"},
        {"role": "user", "content": f"{question} JSON:{json.dumps(records)}"}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    answer = response['choices'][0]['message']['content'] 
    return answer

In [93]:
def answer_question(question):
    query = get_cypher(question)
    print(f"Running cypher to get data.")
    print(query)
    records = get_neo4j_records(query)
    print(f"Found results.")
    print(records)
    answer = format_answer(records, question)
    print(answer)

In [106]:
answer_question("What was the score when Japan played Sweden?")

Running cypher to get data.
MATCH (:Team {name: 'Japan'})-[sg:PLAYED_IN]->(match:Match)<-[sc:PLAYED_IN]-(:Team {name: 'Sweden'}) RETURN sg.score AS JapanScore, sc.score AS SwedenScore
Found results.
[{'JapanScore': 3, 'SwedenScore': 1}, {'JapanScore': 0, 'SwedenScore': 8}, {'JapanScore': 0, 'SwedenScore': 2}]
The score when Japan played Sweden were: 

- In the first match, Japan scored 3 goals and Sweden scored 1 goal.
- In the second match, Japan scored 0 goals and Sweden scored 8 goals.
- In the third match, Japan scored 0 goals and Sweden scored 2 goals.


In [112]:
answer_question("Who won when Denmark played Australia?")

Running cypher to get data.
MATCH (:Team {name: 'Denmark'})-[sg:PLAYED_IN]->(match:Match)<-[sc:PLAYED_IN]-(:Team {name: 'Australia'})
WITH sg.score AS DenmarkScore, sc.score AS AustraliaScore
RETURN 
    CASE 
        WHEN DenmarkScore > AustraliaScore THEN 'Denmark'
        WHEN DenmarkScore < AustraliaScore THEN 'Australia'
        ELSE 'Tie'
    END AS winner
Found results.
[{'winner': 'Denmark'}]
Denmark won when they played against Australia.


In [113]:
answer_question("What was the goal differential when England played Sweden?")

Running cypher to get data.
MATCH (:Team {name: 'England'})-[sg:PLAYED_IN]->(match:Match)<-[ss:PLAYED_IN]-(:Team {name: 'Sweden'})
RETURN sg.score - ss.score AS goalDifferential
Found results.
[{'goalDifferential': -1}]
The goal differential when England played Sweden was -1 goal. This means that Sweden scored 1 more goal than England.


In [114]:
answer_question("Which team had the best goal differential over all matches?")

Running cypher to get data.
MATCH (t:Team)-[s:PLAYED_IN]->(:Match)<-[s2:PLAYED_IN]-(:Team)
WITH t, SUM(s.score - s2.score) AS goalDiff
RETURN t.name AS teamName, goalDiff ORDER BY goalDiff DESC LIMIT 1
Found results.
[{'teamName': 'USA', 'goalDiff': 100}]
The team with the best goal differential over all matches is USA with a goal differential of 100.
